In [5]:
!pip install pyprind

In [28]:
import pyprind
import pandas as pd
import os

basepath = 'aclImdb'

labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 
                      'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], 
                           ignore_index=True)
            pbar.update()
df.columns = ['review', 'sentiment']

C:\Users\PC\AppData\Local\Temp\ipykernel_14848\2283992236.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([[txt, labels[l]]],
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:41


In [29]:
df

,review,sentiment
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1
3,"I saw this film in a sneak preview, and it is ...",1
4,Bill Paxton has taken the true story of the 19...,1
...,...,...
49995,"Towards the end of the movie, I felt it was to...",0
49996,This is the kind of movie that my enemies cont...,0
49997,I saw 'Descent' last night at the Stockholm Fi...,0
49998,Some films that you pick up for a pound turn o...,0


In [30]:
df.to_csv('movie_data.csv', index=False, encoding='utf-8')

# BoW (Bag of Word)

In [43]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

sentence = ['The apple is delicious', 
            'The banana is sweet',
            'one apple and two banana please']
CntVec = CountVectorizer()
bag = CntVec.fit_transform(sentence)

In [44]:
CntVec.vocabulary_

{'the': 8,
 'apple': 1,
 'is': 4,
 'delicious': 3,
 'banana': 2,
 'sweet': 7,
 'one': 5,
 'and': 0,
 'two': 9,
 'please': 6}

In [45]:
tmp = bag.toarray()
tmp

array([[0, 1, 0, 1, 1, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 1, 0, 0, 1, 1, 0],
       [1, 1, 1, 0, 0, 1, 1, 0, 0, 1]], dtype=int64)

In [46]:
tmp.shape

(3, 10)

# TF_IDF (단어빈도-역문서빈도)

In [47]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)

tfidf.fit_transform(tmp).toarray()

array([[0.        , 0.45985353, 0.        , 0.60465213, 0.45985353,
        0.        , 0.        , 0.        , 0.45985353, 0.        ],
       [0.        , 0.        , 0.45985353, 0.        , 0.45985353,
        0.        , 0.        , 0.60465213, 0.45985353, 0.        ],
       [0.44036207, 0.3349067 , 0.3349067 , 0.        , 0.        ,
        0.44036207, 0.44036207, 0.        , 0.        , 0.44036207]])

In [61]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[:25000, 'review'].values
y_test = df.loc[:25000, 'sentiment'].values

In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

tfidf = TfidfVectorizer(strip_accents=None, 
                        lowercase=False, 
                        preprocessor=None,
                        stop_words=None, 
                        ngram_range=(1, 1))
tfidf_lr = Pipeline([('tfidf', tfidf), 
          ('lr', LogisticRegression(solver='liblinear', C=10, penalty='l2', random_state=0))])

In [63]:
tfidf_lr.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer(lowercase=False)),
                ('lr',
                 LogisticRegression(C=10, random_state=0, solver='liblinear'))])

In [64]:
p_test = tfidf_lr.predict(X_test)
p_test

array([1, 1, 1, ..., 0, 0, 1], dtype=int64)

In [67]:
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score

print(f1_score(y_test, p_test))
print(accuracy_score(y_test, p_test))
print(confusion_matrix(y_test, p_test))
# 긍정적인지 부정적인지 만들어본 함수
# 존재만 인식할것

0.9924030387844863
0.9924003039878405
[[12401    99]
 [   91 12410]]
